# создайёт файлы .txt с размерами картинок

In [ ]:
import os
from PIL import Image

# Путь к папке с изображениями
image_folder = r"C:\Users\PC3\YOLO_(26.06)\runs\detect\predict4"

# Путь к папке для сохранения файлов с размерами изображений
output_folder = r"C:\Users\PC3\Desktop\Emelyan_metriks\img_size"

# Создаем папку для выходных файлов, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Проходимся по всем файлам в папке с изображениями
for image_name in os.listdir(image_folder):
    # Проверяем, что файл имеет расширение .jpg
    if image_name.lower().endswith(".jpg"):
        # Полный путь к изображению
        image_path = os.path.join(image_folder, image_name)
        
        # Открываем изображение и получаем его размеры
        with Image.open(image_path) as img:
            width, height = img.size
        
        # Имя файла для записи размеров
        output_file = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}.txt")
        
        # Записываем размеры в файл
        with open(output_file, "w") as f:
            f.write(f"Width: {width}\nHeight: {height}")

print("Готово!")


# Перевод из метрик yolo  в координаты по пиксельно



In [10]:
import os
def perevod_cord(img_size_folder,start_folder,end_folder):
    img_size_folder = img_size_folder

# Путь к папке с разметкой YOLO
    gt_folder = start_folder
# Путь к папке для сохранения новых файлов разметки
    output_folder = end_folder

# Создаем папку для выходных файлов, если она не существует
    os.makedirs(output_folder, exist_ok=True)

# Функция для чтения размеров изображения из файла
    def read_image_size(file_path):
        with open(file_path, "r") as f:
            lines = f.readlines()
            width = int(lines[0].split(":")[1].strip())
            height = int(lines[1].split(":")[1].strip())
        return width, height

# Проходимся по всем файлам в папке с размерами изображений
    for size_file in os.listdir(img_size_folder):
        if size_file.lower().endswith(".txt"):
        # Имя изображения без расширения
            image_name = os.path.splitext(size_file)[0]
        
        # Полный путь к файлу с размерами изображения
            size_file_path = os.path.join(img_size_folder, size_file)
        
        # Получаем размеры изображения
            width, height = read_image_size(size_file_path)
        
        # Полный путь к файлу с разметкой YOLO
            gt_file_path = os.path.join(gt_folder, f"{image_name}.txt")
        
            if os.path.exists(gt_file_path):
            # Читаем разметку YOLO и преобразуем координаты
                with open(gt_file_path, "r") as gt_file:
                    yolo_data = gt_file.readlines()
            
                new_annotations = []
                for line in yolo_data:
                    parts = line.strip().split()
                    class_id = parts[0]
                    x_center = float(parts[1]) * width
                    y_center = float(parts[2]) * height
                    bbox_width = float(parts[3]) * width
                    bbox_height = float(parts[4]) * height
                
                # Вычисляем координаты углов bounding box
                    x_min = x_center - (bbox_width / 2)
                    y_min = y_center - (bbox_height / 2)
                    x_max = x_center + (bbox_width / 2)
                    y_max = y_center + (bbox_height / 2)
                
                # Форматируем строку для новой разметки
                    new_line = f"{class_id} {int(x_min)} {int(y_min)} {int(x_max)} {int(y_max)}"
                    new_annotations.append(new_line)
            
            # Полный путь к файлу для сохранения новой разметки
                output_file_path = os.path.join(output_folder, f"{image_name}.txt")
            
            # Записываем новую разметку в файл
                with open(output_file_path, "w") as output_file:
                    output_file.write("\n".join(new_annotations))

    print("Готово!")

    

In [13]:
perevod_cord('img_size','gt/val','gt_perevod')
perevod_cord('img_size','predict','predict_prevod')
perevod_cord('img_size','pr_Emelyan/val','pr_Emelyan_prevod')

Готово!
Готово!
Готово!


# Расчёт метрик

In [ ]:
import os
from collections import defaultdict

# Путь к папке с истинной разметкой
gt_folder = 'gt_perevod'

# Путь к папке с предсказанной разметкой
predict_folder = 'predict_perevod'

# Словарь для сопоставления идентификаторов классов с их именами
class_names = {
    0: 'птицы',
    1: 'вертолёты',
    2: 'самолёты',
    3: 'дроны',
    4: 'баллун',
    5: 'баллун_с'
}

# Функция для чтения разметки из файла
def read_annotations(file_path):
    annotations = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_min = int(parts[1])
            y_min = int(parts[2])
            x_max = int(parts[3])
            y_max = int(parts[4])
            annotations.append((class_id, x_min, y_min, x_max, y_max))
    return annotations

# Функция для вычисления IoU
def calculate_iou(box1, box2):
    x_min1, y_min1, x_max1, y_max1 = box1
    x_min2, y_min2, x_max2, y_max2 = box2
    
    inter_x_min = max(x_min1, x_min2)
    inter_y_min = max(y_min1, y_min2)
    inter_x_max = min(x_max1, x_max2)
    inter_y_max = min(y_max1, y_max2)
    
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x_max1 - x_min1) * (y_max1 - y_min1)
    box2_area = (x_max2 - x_min2) * (y_max2 - y_min2)
    
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# Порог IoU для определения совпадения объектов
iou_threshold = 0.5

# Словари для хранения статистики
missed_objects = defaultdict(int)
false_positives = defaultdict(int)
iou_sums = defaultdict(float)
iou_counts = defaultdict(int)
classification_errors = defaultdict(int)

# Проходимся по всем файлам в папке с истинной разметкой
for gt_file in os.listdir(gt_folder):
    if gt_file.lower().endswith(".txt"):
        # Полный путь к файлам с разметкой
        gt_file_path = os.path.join(gt_folder, gt_file)
        predict_file_path = os.path.join(predict_folder, gt_file)
        
        if os.path.exists(predict_file_path):
            # Читаем разметку
            gt_annotations = read_annotations(gt_file_path)
            predict_annotations = read_annotations(predict_file_path)
            
            # Переменные для учета найденных и пропущенных объектов
            matched_gt = set()
            matched_pred = set()
            
            # Сравниваем предсказания с истинной разметкой
            for gt_idx, gt_ann in enumerate(gt_annotations):
                gt_class_id, gt_box = gt_ann[0], gt_ann[1:]
                best_iou = 0
                best_pred_idx = -1
                for pred_idx, pred_ann in enumerate(predict_annotations):
                    if pred_idx in matched_pred:
                        continue
                    pred_class_id, pred_box = pred_ann[0], pred_ann[1:]
                    iou = calculate_iou(gt_box, pred_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_pred_idx = pred_idx
                
                if best_iou >= iou_threshold:
                    matched_gt.add(gt_idx)
                    matched_pred.add(best_pred_idx)
                    if gt_class_id == predict_annotations[best_pred_idx][0]:
                        iou_sums[gt_class_id] += best_iou
                        iou_counts[gt_class_id] += 1
                    else:
                        classification_errors[gt_class_id] += 1
                        false_positives[predict_annotations[best_pred_idx][0]] += 1
                else:
                    missed_objects[gt_class_id] += 1
            
            # Учет выдуманных объектов
            for pred_idx, pred_ann in enumerate(predict_annotations):
                if pred_idx not in matched_pred:
                    false_positives[pred_ann[0]] += 1

# Вычисляем итоговую статистику
total_gt_objects = sum(missed_objects.values()) + sum(iou_counts.values()) + sum(classification_errors.values())
total_pred_objects = sum(false_positives.values()) + sum(iou_counts.values()) + sum(classification_errors.values())

missed_objects_percentage = {cls_id: (count / total_gt_objects) * 100 for cls_id, count in missed_objects.items()}
false_positives_percentage = {cls_id: (count / total_pred_objects) * 100 for cls_id, count in false_positives.items()}
average_iou_per_class = {cls_id: (iou_sums[cls_id] / iou_counts[cls_id]) if iou_counts[cls_id] > 0 else 0 for cls_id in iou_counts.keys()}
classification_errors_percentage = {cls_id: (count / total_gt_objects) * 100 for cls_id, count in classification_errors.items()}

# Выводим результаты
print("Количество пропущенных объектов:")
for cls_id, count in missed_objects.items():
    print(f"Класс {class_names[cls_id]}: {count} ({missed_objects_percentage[cls_id]:.2f}%)")

print("\nКоличество выдуманных объектов:")
for cls_id, count in false_positives.items():
    print(f"Класс {class_names[cls_id]}: {count} ({false_positives_percentage[cls_id]:.2f}%)")

print("\nСреднее значение IoU по классам:")
for cls_id, avg_iou in average_iou_per_class.items():
    print(f"Класс {class_names[cls_id]}: {avg_iou:.4f}")

print("\nОшибки классификации по классам:")
for cls_id, count in classification_errors.items():
    print(f"Класс {class_names[cls_id]}: {count} ({classification_errors_percentage[cls_id]:.2f}%)")

# Общее количество пропущенных и выдуманных объектов
total_missed_objects = sum(missed_objects.values())
total_false_positives = sum(false_positives.values())

total_missed_percentage = (total_missed_objects / total_gt_objects) * 100 if total_gt_objects > 0 else 0
total_false_positives_percentage = (total_false_positives / total_pred_objects) * 100 if total_pred_objects > 0 else 0

print("\nОбщее количество пропущенных объектов:")
print(f"Всего пропущено: {total_missed_objects} ({total_missed_percentage:.2f}%)")

print("\nОбщее количество выдуманных объектов:")
print(f"Всего выдумано: {total_false_positives} ({total_false_positives_percentage:.2f}%)")

# Общее количество обнаруженных объектов
total_detected_objects = sum(iou_counts.values())

print("\nОбщее количество обнаруженных объектов:")
print(f"Всего обнаружено: {total_detected_objects}")

# Количество обнаруженных объектов для каждого класса
print("\nКоличество обнаруженных объектов по классам:")
for cls_id, count in iou_counts.items():
    print(f"Класс {class_names[cls_id]}: {count}")
